In [11]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('../database/pseudo_observations/vas.txt')
data1 = np.array(data1.iloc[::2, :])
data1 = np.array([float(i[0]) for i in data1])

data2 = pd.read_csv('../database/pseudo_observations/LQD.txt')
data2 = np.array(data2.iloc[::2, :])
data2 = np.array([float(i[0]) for i in data2])

data3 = pd.read_csv('../database/pseudo_observations/HYG.txt')
data3 = np.array(data3.iloc[::2, :])
data3 = np.array([float(i[0]) for i in data3])

data4 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data4 = np.array(data4.iloc[::2, :])
data4 = np.array([float(i[0]) for i in data4])

data5 = pd.read_csv('../database/pseudo_observations/IVV.txt')
data5 = np.array(data5.iloc[::2, :])
data5 = np.array([float(i[0]) for i in data5])

data6 = pd.read_csv('../database/pseudo_observations/GLD.txt')
data6 = np.array(data6.iloc[::2, :])
data6 = np.array([float(i[0]) for i in data6])

data7 = pd.read_csv('../database/pseudo_observations/1321.txt')
data7 = np.array(data7.iloc[::2, :])
data7 = np.array([float(i[0]) for i in data7])

data8 = pd.read_csv('../database/pseudo_observations/AUD.txt')
data8 = np.array(data8.iloc[::2, :])
data8 = np.array([float(i[0]) for i in data8])

data9 = pd.read_csv('../database/pseudo_observations/EUR.txt')
data9 = np.array(data9.iloc[::2, :])
data9 = np.array([float(i[0]) for i in data9])

data10 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data10 = np.array(data10.iloc[::2, :])
data10 = np.array([float(i[0]) for i in data10])

In [15]:
import pyvinecopulib as pv
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian,
                                             pv.BicopFamily.clayton,
                                             pv.BicopFamily.frank,
                                             pv.BicopFamily.gumbel,
                                             pv.BicopFamily.bb1])

u = np.stack([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10]).T
cop = pv.Vinecop(u, controls=controls)

u_sim = cop.simulate(100)
np.savetxt("u_sim.txt", u_sim, delimiter=",")
print(u_sim)

[[0.32350916 0.1270862  0.16654424 0.15013747 0.24370557 0.02917075
  0.13012259 0.76447731 0.19739071 0.15102285]
 [0.14173509 0.19042953 0.25526832 0.17832706 0.40329391 0.09701876
  0.6843499  0.96106437 0.36336614 0.17842631]
 [0.85466427 0.92222173 0.91760671 0.92685667 0.86298837 0.92099135
  0.3436603  0.14175098 0.95298566 0.92606388]
 [0.47856116 0.38149205 0.17914345 0.25605926 0.29498633 0.52622566
  0.65724037 0.82387006 0.34146369 0.25524672]
 [0.66908468 0.89219959 0.83311691 0.91765162 0.87605981 0.91554471
  0.25315453 0.13079432 0.93371005 0.91465493]
 [0.36292724 0.19487358 0.29063668 0.2492165  0.27529286 0.13485268
  0.25051407 0.19480884 0.16670735 0.24508604]
 [0.76669364 0.62012003 0.60030401 0.59705214 0.62483752 0.16805745
  0.44349058 0.47479775 0.41793655 0.60792099]
 [0.22714789 0.17379985 0.18379308 0.22450822 0.32469872 0.39273262
  0.21732433 0.61490254 0.22649122 0.2183156 ]
 [0.66198976 0.80817946 0.82963309 0.83844136 0.87048531 0.60874212
  0.65995906

In [4]:
cop.structure

<pyvinecopulib.RVineStructure>
5 5 3 2 6 2 2 4 10 10 
1 3 2 4 2 4 4 10 4 
3 2 4 6 4 8 10 2 
2 4 6 9 8 10 8 
4 6 9 8 10 6 
6 9 8 10 9 
9 8 10 3 
8 10 5 
10 1 
7 

In [5]:
from database.database_parser import create_connection
from database.database_parser import select_curr_solution

test_connection = create_connection('../database/abacus_database.db')

In [6]:
from database.database_parser import select_price_data

def generate_volatility(params, data):
    n_observations = len(data)
    initial_squared_vol_estimate = (params[0]
                                        + params[1] * (data[0] ** 2)
                                        + params[3] * (data[0] ** 2) * np.where(data[0] < 0, 1, 0)
                                        + params[2] * (data[0] ** 2))
    current_squared_vol_estimate = initial_squared_vol_estimate

    for i in range(0, n_observations):
        current_squared_vol_estimate = (params[0] + params[1] * (data[i - 1] ** 2)
                                            + params[3] * (data[i - 1] ** 2) * np.where(data[i - 1] < 0, 1, 0)
                                            + params[2] * current_squared_vol_estimate)

    return np.sqrt(current_squared_vol_estimate)

def get_volatility(asset: str):
    opt_sol = select_curr_solution(test_connection, asset)
    adj_close = select_price_data(test_connection, '1321')
    log_returns = np.array(np.log(adj_close / adj_close.shift(1))[1:])
    return generate_volatility(params=opt_sol, data=log_returns)

In [7]:
from distributions.student_poisson_mixture import spm
# Calculate all inverse samples.
def generate_samples(us, asset):
    opt_sol = select_curr_solution(test_connection, asset)
    mu = opt_sol[4]
    sigma = get_volatility(asset)[0]
    kappa = opt_sol[5]
    lamb = opt_sol[6]
    nu = opt_sol[7]

    for p in us:
        s = spm.ppf(p, mu, sigma, kappa, lamb, nu)
        print(s.root)

generate_samples(u_sim[:, 4], 'IVV')

1.0584664836470876
0.29376358048548606
-0.018547013236331327
-0.018540169082710785
0.9373604137841106
-0.009370369113802086
-0.11197432799522992
-0.014153200520035035
0.01968628690954487
0.048521438908058705
-0.034856268800176095
-0.018702032196939347
0.009192375284793185
-0.04457269369490421
0.5549821022052526
-0.030350200842128368
0.3424838732209676
0.04803011991686046
-0.07242956460401717
0.029592015512437717
0.061801177424709794
0.12858739572443773
-0.04321951457804534
-0.042140300131046894
0.07754664967959247
-0.044797028327946076
0.09282195310411946
-0.019354058068346197
0.07639690173581754
-0.06956005699463562
-0.019735101098384895
0.04194976724074194
0.09362345430676913
-0.04692863344541148
-0.025877356333441622
0.07485770663120202
-0.5855520035975932
0.8069473200891641
0.013652933819617517
-0.04390634923081404
0.014466340724592618
-0.009486637105158043
-0.006765109687165978
-0.013501740080299217
0.18576803323188584
-0.04282738737880735
0.05019180971988053
0.06142885116119932
0

ValueError: f(a) and f(b) must have different signs

In [10]:
df_ret = pd.read_csv('../database/return_samples/IVV.txt')
len(df_ret)

76